In [1]:
import pandas as pd
import numpy as np
import os

In [5]:
## RUN THIS ONLY ONCE WHEN YOU HAVE NEW DATA

# # Function to add the feature descriptions as column names to a DataFrame
# def add_feature_descriptions(speaker_df, descriptions_df):
#     # Set the new column headers
#     descriptions_df.columns = ['feature_name', 'feature_description']
#     # Use the 'feature_name' column as the column names for speaker_df
#     speaker_df.columns = descriptions_df['feature_name'].values
#     return speaker_df


# # Function to add the labels to the speaker DataFrame
# def add_labels(speaker_df, labels_df, participant_id):
#     # Find the matching row in the labels DataFrame by participant ID
#     labels_row = labels_df.loc[labels_df['Participant_ID'] == participant_id]
#     # If the row is found, add the Depression and Gender values as new columns
#     if not labels_row.empty:
#         speaker_df['Depression'] = labels_row['Depression'].values[0]
#         speaker_df['Gender'] = labels_row['Gender'].values[0]
#         speaker_df['Participant_ID'] = participant_id
#     return speaker_df

# # Paths to CSV files
features_description_path = '/Users/Ayan/Desktop/CS5622/HW5/feature_description.csv'
labels_path = '/Users/Ayan/Desktop/CS5622/HW5/labels.csv'
features_train_dir = '/Users/Ayan/Desktop/CS5622/HW5/features_train'

# # Read the feature descriptions and labels CSV files
features_description_df = pd.read_csv(features_description_path, encoding='latin-1', header=None)
labels_df = pd.read_csv(labels_path, encoding='latin-1')

# # Iterate over each speaker CSV file in the features_train directory
# for file in os.listdir(features_train_dir):
#     if file.endswith('.csv'):
#         # Extract the participant ID from the file name
#         participant_id = int(file.split('_')[1].split('.')[0])
        
#         # Read the speaker CSV file
#         speaker_df = pd.read_csv(os.path.join(features_train_dir, file), encoding='latin-1')
        
#         # Add feature descriptions as column headers
#         speaker_df = add_feature_descriptions(speaker_df, features_description_df)
        
#         # Add labels based on participant ID
#         speaker_df = add_labels(speaker_df, labels_df, participant_id)
                
#         # make sure the dataframe is updated and the file itself is updated too
#         speaker_df.to_csv(os.path.join(features_train_dir, file), index=False)


In [11]:
# function to aggregate all training csv's into one big training data table
def aggregate_training_data(features_train_dir):
    # Create an empty DataFrame to store the aggregated data
    aggregated_df = pd.DataFrame()
    
    # Iterate over each speaker CSV file in the features_train directory
    for file in os.listdir(features_train_dir):
        if file.endswith('.csv'):
            # Read the speaker CSV file
            speaker_df = pd.read_csv(os.path.join(features_train_dir, file), encoding='latin-1')
            # Append the speaker data to the aggregated DataFrame
            aggregated_df = pd.concat([aggregated_df, speaker_df], ignore_index=True)
    
    return aggregated_df


# Aggregate all training data into one DataFrame
training_data = aggregate_training_data(features_train_dir)

# Save the aggregated training data to a CSV file
training_data.to_csv('/Users/Ayan/Desktop/CS5622/HW5/training_data.csv', index=False)

### RNN For Depression Detection

In [ ]:
# read in the training data
training_data = pd.read_csv('/Users/Ayan/Desktop/CS5622/HW5/training_data.csv', encoding='latin-1')

# print the first 5 rows of the training data
print(training_data.head())